In [ ]:
import pandas as pd

#Tableau CSVs adjustments
#Load the 3 exported CSVs
lr_results = pd.read_csv("lr_results.csv")
rf_results = pd.read_csv("rf_results.csv")
xgb_results = pd.read_csv("xgb_results.csv")

#Merge on CustomerId + Actual
combined_results = lr_results.merge(rf_results, on=["CustomerId", "Actual"])
combined_results = combined_results.merge(xgb_results, on=["CustomerId", "Actual"])

#Save final dataset for Tableau
combined_results.to_csv("bank_churn_model_comparison.csv", index=False)

print("Combined results saved as bank_churn_model_comparison.csv")


In [ ]:
#tableau data
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

df = pd.read_csv("bank_churn_model_comparison.csv")

models = {
    "LR": ("LR_pred", "LR_prob"),
    "RF": ("RF_pred", "RF_prob"),
    "XGB": ("XGB_pred", "XGB_prob")
}

metrics = []
roc_rows = []

for name, (pred_col, prob_col) in models.items():
    y_true = df["Actual"].values
    y_pred = df[pred_col].values
    y_prob = df[prob_col].values

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_prob)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    metrics.append({
        "Model": name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "ROC_AUC": auc,
        "TN": tn, "FP": fp, "FN": fn, "TP": tp
    })

    fpr, tpr, thresh = roc_curve(y_true, y_prob)
    for f, t, th in zip(fpr, tpr, thresh):
        roc_rows.append({"Model": name, "FPR": f, "TPR": t, "Threshold": th})

pd.DataFrame(metrics).to_csv("model_metrics.csv", index=False)
pd.DataFrame(roc_rows).to_csv("roc_coords.csv", index=False)
print("Saved model_metrics.csv and roc_coords.csv")
